In [ ]:
# Using the SQL database approach, compare
# manta data with model results, varying
# the averaging period and the max age.


In [5]:
import os
import time
from collections import defaultdict
import six

import logging as log
import glob
import re

from matplotlib import colors, cm
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from matplotlib.colors import LogNorm

import numpy as np

import pandas as pd
import xarray as xr

from sqlite3 import dbapi2 as sql

from stompy.grid import unstructured_grid
from stompy import utils, memoize
from stompy.spatial import proj_utils 
from stompy.plot import plot_wkb
from stompy.model import data_comparison
import stompy.plot.cmap as scmap

import postprocess_v00 as post

%matplotlib notebook

In [6]:
ll2utm=proj_utils.mapper('WGS84','EPSG:26910')

In [7]:
import sql_common
six.moves.reload_module(sql_common)


# try to run everything that can be cached and re-used through
# here. 
PtmSet=sql_common.PtmSet


In [3]:
dbs=glob.glob("/opt2/sfb_ocean/ptm/all_source/*/ptm_and_grid.db")
dbs.sort()

ptm_set=PtmSet(databases=dbs)

In [18]:
# Load the manta data, clean it up
# This is maybe out of date - it has dups, blanks, and more missing x,y
# than what I expect.
# v01nofiber has 1 sample missing lat/lon
# v01std has 3... but those are LabBlanks. why did they get through?
# one issue is that there are some no-fiber stations where the only
# thing found was fibers, so those just go away in the nofiber case.
# should fix that!

manta_std=pd.read_csv('manta_summary-v01std.csv')
manta_nofiber=pd.read_csv('manta_summary-v01nofiber.csv')

split_fields=('SampleID','part_per_m2','part_per_m3','area_m2')
common_fields=('SampleID','SampleType','x','y','Season','SampleDate')
manta_combined=pd.merge(manta_std.loc[:, split_fields],
                        manta_nofiber.loc[:, split_fields],
                        left_on='SampleID',right_on='SampleID',
                        how='outer',suffixes=['_std','_nofiber'])
manta=pd.merge(manta_nofiber.loc[:,common_fields],
               manta_combined,
               left_on='SampleID',right_on='SampleID',how='outer')

assert np.all(np.where( ~( manta['area_m2_std'].isnull() | manta['area_m2_nofiber'].isnull()),
                        manta['area_m2_std']==manta['area_m2_nofiber'],
                        True))

area=np.where(manta['area_m2_std'].isnull(),
              manta['area_m2_nofiber'],
              manta['area_m2_std'])
manta['area_m2']=area

# A few lab blanks are in there - they have zero area
manta=manta.
manta.head()

,SampleID,SampleType,x,y,Season,SampleDate,part_per_m2_std,part_per_m3_std,area_m2_std,part_per_m2_nofiber,part_per_m3_nofiber,area_m2_nofiber,area_m2
0,CB-4-Manta-21Aug2017,Trawl,550326.816800,4.197530e+06,Dry,2017-08-21,0.029667,0.312288,2118.727190,0.001180,0.012421,2118.727190,2118.727190
1,CB4-Manta-16Nov2017,Trawl,550326.816800,4.197530e+06,Wet,2017-11-16,NaN,NaN,NaN,0.579936,6.104594,1081.727744,1081.727744
2,CB5-Manta-16Nov2017,Trawl,552177.481202,4.188548e+06,Wet,2017-11-16,NaN,NaN,NaN,0.178455,1.878478,1408.382016,1408.382016
3,CB5-Manta-22Aug2017,Trawl,552177.481202,4.188548e+06,Dry,2017-08-22,NaN,NaN,NaN,0.040986,0.431429,1447.658557,1447.658557
4,CB5-Manta-5Nov2017,Trawl,552177.481202,4.188548e+06,Dry,2017-11-05,0.299179,3.149252,1866.537295,0.030895,0.325211,1866.537295,1866.537295


In [37]:
manta.loc[ manta.area_m2==0, : ]

,SampleID,SampleType,x,y,Season,SampleDate,part_per_m2_std,part_per_m3_std,area_m2_std,part_per_m2_nofiber,part_per_m3_nofiber,area_m2_nofiber,area_m2
13,CB9-Manta-Blank-11Jan18,FieldBlank,561567.365255,4.172290e+06,Wet,2018-01-11,NaN,NaN,NaN,NaN,NaN,0.0,0.0
21,GFNMS25-Manta-Blank-11Sept2017,FieldBlank,505534.726369,3.500192e+06,Dry,2017-09-11,inf,inf,0.0,inf,inf,0.0,0.0
30,LSB14-Manta-Blank-06Mar2018,FieldBlank,581512.786788,4.148595e+06,Wet,2018-03-06,NaN,NaN,NaN,inf,inf,0.0,0.0
39,MBNMS29-Manta-BLANK-11Jan18,FieldBlank,544773.990739,4.184378e+06,Wet,2018-01-11,inf,inf,0.0,NaN,NaN,0.0,0.0
54,SB13-Manta-BLANK-23Aug2017,FieldBlank,570632.092412,4.157015e+06,Dry,2017-08-23,inf,inf,0.0,inf,inf,0.0,0.0
60,SPB3-Manta-Blank-21Aug2017,FieldBlank,557262.007074,4.209491e+06,Dry,2017-08-21,inf,inf,0.0,inf,inf,0.0,0.0
65,Manta_LabBlank1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
66,Manta_LabBlank2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
67,Manta_LabBlank3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0


In [10]:
# Developing the SQL query for point data.

In [9]:
ptm_set.databases

['/opt2/sfb_ocean/ptm/all_source/20170715/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20170815/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20170915/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20171015/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20171115/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20171215/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20180115/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20180215/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20180315/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20180415/ptm_and_grid.db',
 '/opt2/sfb_ocean/ptm/all_source/20180515/ptm_and_grid.db']

In [12]:
con=ptm_set.db_to_con(ptm_set.databases[1])

curs=con.cursor()

In [19]:
# try querying for a single point.
# given how long the trawls are, and 
rec=manta.iloc[0,:]
rec

SampleID               CB-4-Manta-21Aug2017
SampleType                            Trawl
x                                    550327
y                               4.19753e+06
Season                                  Dry
SampleDate                       2017-08-21
part_per_m2_std                   0.0296674
part_per_m3_std                    0.312288
area_m2_std                         2118.73
part_per_m2_nofiber              0.00117995
part_per_m3_nofiber               0.0124206
area_m2_nofiber                     2118.73
area_m2                             2118.73
Name: 0, dtype: object

In [23]:
rec=rec.copy()
rec['new_field']=10


In [28]:
results=[]
for idx,rec in manta.iterrows():
    rec=rec.copy()
    t_sample=np.datetime64(rec.SampleDate)
    t_start=t_sample-np.timedelta64(2,'D')
    t_stop =t_sample+np.timedelta64(2,'D')

    c_select=ptm_set.grid().select_cells_nearest(np.r_[rec.x,rec.y])
    rec['cell']=c_select
    
    for loads in ['std','nofiber']:
        ds=ptm_set.conc_query(t_start=t_start,
                              t_stop=t_stop,
                              z_filter='surf',
                              max_age=np.timedelta64(5,'D'),
                              loads=loads)
        smooth_c=ptm_set.smooth(ds.conc.values)
        rec['pred_'+loads]=smooth_c[c_select]
    results.append(rec)

Trying: /opt2/sfb_ocean/ptm/all_source/20170715/ptm_and_grid.db
memory table wasn't there.  no worries.
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
                 ptm_group as grp, load.std as load
            where loc.time>=1503100800 and loc.time < 1503446400
              and loc.particle_id=p.id
              and p.release_id=rel.id
              and rel.group_id=grp.id
              and grp.name=load.group_name 
              and loc.cell>=0
              and (loc.time-rel.time)<432000
              and loc.z_from_surface>-0.5
              
            group by loc.cell
DB:  /opt2/sfb_ocean/ptm/all_source/20170715/ptm_and_grid.db
Query time to return data 113.62s
Trying: /opt2/sfb_ocean/ptm/all_source/20170815/ptm_and_grid.db
memory table wasn't there.  no worries.
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
           

Query time to return data 0.04s


INFO:utils:49812/99089
INFO:utils:98007/99089


Trying: /opt2/sfb_ocean/ptm/all_source/20170715/ptm_and_grid.db
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
                 ptm_group as grp, load.nofiber as load
            where loc.time>=1503100800 and loc.time < 1503446400
              and loc.particle_id=p.id
              and p.release_id=rel.id
              and rel.group_id=grp.id
              and grp.name=load.group_name 
              and loc.cell>=0
              and (loc.time-rel.time)<432000
              and loc.z_from_surface>-0.5
              
            group by loc.cell
DB:  /opt2/sfb_ocean/ptm/all_source/20170715/ptm_and_grid.db
Query time to return data 50.83s
Trying: /opt2/sfb_ocean/ptm/all_source/20170815/ptm_and_grid.db
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
              

Trying: /opt2/sfb_ocean/ptm/all_source/20170715/ptm_and_grid.db
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
                 ptm_group as grp, load.std as load
            where loc.time>=1510617600 and loc.time < 1510963200
              and loc.particle_id=p.id
              and p.release_id=rel.id
              and rel.group_id=grp.id
              and grp.name=load.group_name 
              and loc.cell>=0
              and (loc.time-rel.time)<432000
              and loc.z_from_surface>-0.5
              
            group by loc.cell
DB:  /opt2/sfb_ocean/ptm/all_source/20170715/ptm_and_grid.db
Query time to return data 20.58s
Trying: /opt2/sfb_ocean/ptm/all_source/20170815/ptm_and_grid.db
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
                 p

...

Trying: /opt2/sfb_ocean/ptm/all_source/20180115/ptm_and_grid.db
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
                 ptm_group as grp, load.nofiber as load
            where loc.time>=1521244800 and loc.time < 1521590400
              and loc.particle_id=p.id
              and p.release_id=rel.id
              and rel.group_id=grp.id
              and grp.name=load.group_name 
              and loc.cell>=0
              and (loc.time-rel.time)<432000
              and loc.z_from_surface>-0.5
              
            group by loc.cell
DB:  /opt2/sfb_ocean/ptm/all_source/20180115/ptm_and_grid.db
Query time to return data 0.00s
Trying: /opt2/sfb_ocean/ptm/all_source/20180215/ptm_and_grid.db
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
               

Query time to return data 0.06s
Trying: /opt2/sfb_ocean/ptm/all_source/20180315/ptm_and_grid.db
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
                 ptm_group as grp, load.std as load
            where loc.time>=1503273600 and loc.time < 1503619200
              and loc.particle_id=p.id
              and p.release_id=rel.id
              and rel.group_id=grp.id
              and grp.name=load.group_name 
              and loc.cell>=0
              and (loc.time-rel.time)<432000
              and loc.z_from_surface>-0.5
              
            group by loc.cell
DB:  /opt2/sfb_ocean/ptm/all_source/20180315/ptm_and_grid.db
Query time to return data 0.00s
Trying: /opt2/sfb_ocean/ptm/all_source/20180415/ptm_and_grid.db
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_rele

Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
                 ptm_group as grp, load.nofiber as load
            where loc.time>=1503273600 and loc.time < 1503619200
              and loc.particle_id=p.id
              and p.release_id=rel.id
              and rel.group_id=grp.id
              and grp.name=load.group_name 
              and loc.cell>=0
              and (loc.time-rel.time)<432000
              and loc.z_from_surface>-0.5
              
            group by loc.cell
DB:  /opt2/sfb_ocean/ptm/all_source/20180415/ptm_and_grid.db
Query time to return data 0.00s
Trying: /opt2/sfb_ocean/ptm/all_source/20180515/ptm_and_grid.db
Query

           select min(loc.cell),sum(load.part_per_m3 * rel.volume / rel.count )
            from particle_loc as loc, particle as p, ptm_release as rel, 
                 ptm_group as grp, load.nofiber as load
            where loc.t

Trying: /opt2/sfb_ocean/ptm/all_source/20180115/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20180215/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20180315/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20180415/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20180515/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20170715/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20170815/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20170915/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20171015/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20171115/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20171215/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20180115/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20180215/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20180315/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/20180415/ptm_and_grid.db
Trying: /opt2/sfb_ocean/ptm/all_source/2

RTreeError: Coordinates must not have minimums more than maximums

In [31]:
rec

SampleID               SFBay9-18-Manta-18Sept2017
SampleType                                  Trawl
x                                             NaN
y                                             NaN
Season                                        Dry
SampleDate                             2017-09-18
part_per_m2_std                               NaN
part_per_m3_std                               NaN
area_m2_std                                   NaN
part_per_m2_nofiber                      0.039157
part_per_m3_nofiber                      0.412179
area_m2_nofiber                            1004.5
area_m2                                    1004.5
Name: 55, dtype: object